<h2>--- Day 14: Extended Polymerization ---</h2>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2021FSharp/main?urlpath=lab%2Ftree%2FDay14.ipynb)

<p>The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has <a href="https://en.wikipedia.org/wiki/Polymerization" target="_blank">polymerization</a> equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.</p>
<p>The submarine manual contains <span title="HO&#xa;&#xa;HO -&gt; OH">instructions</span> for finding the optimal polymer formula; specifically, it offers a <em>polymer template</em> and a list of <em>pair insertion</em> rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.</p>
<p>For example:</p>
<pre><code>NNCB

CH -&gt; B
HH -&gt; N
CB -&gt; H
NH -&gt; C
HB -&gt; C
HC -&gt; B
HN -&gt; C
NN -&gt; C
BH -&gt; H
NC -&gt; B
NB -&gt; B
BN -&gt; B
BB -&gt; N
BC -&gt; B
CC -&gt; N
CN -&gt; C
</code></pre>
<p>The first line is the <em>polymer template</em> - this is the starting point of the process.</p>
<p>The following section defines the <em>pair insertion</em> rules. A rule like <code>AB -&gt; C</code> means that when elements <code>A</code> and <code>B</code> are immediately adjacent, element <code>C</code> should be inserted between them. These insertions all happen simultaneously.</p>
<p>So, starting with the polymer template <code>NNCB</code>, the first step simultaneously considers all three pairs:</p>
<ul>
<li>The first pair (<code>NN</code>) matches the rule <code>NN -&gt; C</code>, so element <code><em>C</em></code> is inserted between the first <code>N</code> and the second <code>N</code>.</li>
<li>The second pair (<code>NC</code>) matches the rule <code>NC -&gt; B</code>, so element <code><em>B</em></code> is inserted between the <code>N</code> and the <code>C</code>.</li>
<li>The third pair (<code>CB</code>) matches the rule <code>CB -&gt; H</code>, so element <code><em>H</em></code> is inserted between the <code>C</code> and the <code>B</code>.</li>
</ul>
<p>Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.</p>
<p>After the first step of this process, the polymer becomes <code>N<em>C</em>N<em>B</em>C<em>H</em>B</code>.</p>
<p>Here are the results of a few steps using the above rules:</p>
<pre><code>Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
</code></pre>
<p>This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, <code>B</code> occurs 1749 times, <code>C</code> occurs 298 times, <code>H</code> occurs 191 times, and <code>N</code> occurs 865 times; taking the quantity of the most common element (<code>B</code>, 1749) and subtracting the quantity of the least common element (<code>H</code>, 161) produces <code>1749 - 161 = <em>1588</em></code>.</p>
<p>Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. <em>What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?</em></p>

In [ ]:
let input = File.ReadAllText(@"input/14.txt").Split(Environment.NewLine + Environment.NewLine)

let template = input.[0]
let insertionRules = 
    input.[1].Split(Environment.NewLine)
    |> Seq.map (fun line -> line.Split(" -> "))
    |> Seq.map (fun split -> (split.[0].[0], split.[0].[1]), split.[1] |> char)
    |> Map

let templateMap = 
    template
    |> Seq.pairwise
    |> Seq.countBy id
    |> Seq.map (fun (key, count) -> key, bigint count)
    |> Map

In [ ]:
let step insertionRules (templateMap: Map<char * char, bigint>) =
    let expand (key, value) = 
        let insert = insertionRules |> Map.find key
        [
            (fst key, insert), value
            (insert, snd key), value
        ]
    
    templateMap
    |> Map.toSeq
    |> Seq.map expand
    |> Seq.concat
    |> Seq.groupBy (fun (key, count) -> key)
    |> Seq.map (fun (key, counts) -> 
        key, 
        counts 
        |> Seq.map (fun (_, count) -> count)
        |> Seq.sum)
    |> Map


In [ ]:
let rec steps insertionRules templateMap = 
    seq {
        yield templateMap
        yield! 
            templateMap
            |> step insertionRules
            |> steps insertionRules
    }

In [ ]:
let countsByChar lastChar (templateMap: Map<char * char, bigint>) = 
    templateMap
    |> Map.toSeq
    |> Seq.map (fun (key, count) -> fst key, count)
    |> Seq.append [(lastChar, 1)]
    |> Seq.groupBy (fun (key, count) -> key)
    |> Seq.map (fun (key, counts) -> 
        key, 
        counts 
        |> Seq.map (fun (_, count) -> count)
        |> Seq.sum)
    |> Map

In [ ]:
#!time
templateMap
|> steps insertionRules
|> Seq.item 10
|> countsByChar (template |> Seq.last)
|> fun charCounts -> 
    charCounts
    |> Map.values
    |> Seq.min
    |> (-) (charCounts
        |> Map.values
        |> Seq.max)

2321

Wall time: 36.7747ms

<h2 id="part2">--- Part Two ---</h2>

<p>The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of <em>40 steps</em> should do it.</p>
<p>In the above example, the most common element is <code>B</code> (occurring <code>2192039569602</code> times) and the least common element is <code>H</code> (occurring <code>3849876073</code> times); subtracting these produces <code><em>2188189693529</em></code>.</p>
<p>Apply <em>40</em> steps of pair insertion to the polymer template and find the most and least common elements in the result. <em>What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?</em></p>

In [ ]:
#!time
templateMap
|> steps insertionRules
|> Seq.item 40
|> countsByChar (template |> Seq.last)
|> fun charCounts -> 
    charCounts
    |> Map.values
    |> Seq.min
    |> (-) (charCounts
        |> Map.values
        |> Seq.max)

2399822193707

Wall time: 21.5825ms

[Prev](Day13.ipynb) | [Next](Day15.ipynb)